<a href="https://colab.research.google.com/github/juyeonyoon/RecSys/blob/main/02_%E1%84%8F%E1%85%A5%E1%86%AB%E1%84%90%E1%85%A6%E1%86%AB%E1%84%8E%E1%85%B3%E1%84%80%E1%85%B5%E1%84%87%E1%85%A1%E1%86%AB_TF_IDF_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- kss는 한국어 문장 분리 모듈로, 자세한 내용은 [링크](https://pypi.org/project/kss/)를 참고하세요.

In [ ]:
!pip install kss ## 한국어 문장 분리 모듈(문장 단위로 쪼개고 싶을때)

     |████████████████████████████████| 42.4 MB 2.2 MB/s 
     |████████████████████████████████| 175 kB 68.9 MB/s 
  Created wheel for kss: filename=kss-3.4.2-py3-none-any.whl size=42448069 sha256=00eb339eb1a3867227f72e98cf6fae5aadaecf69b34a45faed3733d7ae646867
  Stored in directory: /root/.cache/pip/wheels/ef/22/aa/6399b60516a067ec97fa6599fb2d472aeb25e3f9ee6dae3224
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=7aa7e9c44468ed0ea571e33c5ea0df6867546b4f7414fb08ba43b697d9fa6dd4
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built kss emoji


In [ ]:
import os
import kss
import re

In [ ]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 54.8 MB/s 
Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.co

### Read dataset
- Github의 data 디렉토리에서 news_sample.txt 파일을 다운받아서 분석에 사용해주세요.

In [ ]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수 ## -> 숫자가 있으면 다 num으로 바꿔준다. 
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
# 각자 작업 환경에 맞는 경로를 지정해주세요. Google Colab과 Jupyter환경에서 경로가 다를 수 있습니다.
data_path = '/content/drive/MyDrive/fastcampus/fastcampus-RecSys-master'
all_sentences = []
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding='utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"--- 문서 {idx} 번 ---")
    preprocessed = cleansing_numbers(line)  ## 분석에 필요없는 부분을 날려버리겠다. 
    preprocessed = cleansing_special(preprocessed)
    # 명사만 추출하기 ## -> 조사들은 큰 의미가 없다. 명사(keyword) 추출 (macab 사용)
    preprocessed_news = ' '.join(list(set(mecab.nouns(preprocessed))))
    # 문장 분리해서 사용하는 방법 
    # preprocessed_news = ' '.join(kss.split_sentences(preprocessed))
    print(preprocessed_news)
    all_sentences.append(preprocessed_news)

--- 문서 0 번 ---
표출 도시 고유 지혜 한미 관계 지방 취임 오바마 민족 업그레이드 인사 얼마 올해 추진 과제 출발 단체 사람 슬기 후보 변화 열차 경제 평화 실행 기반 모색 조기 정치 성과 현안 쇠 어제 변수 자질 발전 구체 정부 낭비성 점 개성 기대 바람 활력 때 시간 자본 통일 유치 열쇠 몸 민 한국 논리 바다 바탕 근거 목적 당국 화해 자생력 정치력 전력 유능 방향 동계 자신 정상 방법 회생 속 중단 통행 도 증오 남북 공감대 강원도 미래 기억 축년 능력 흑인 파악 시대 욕 경의선 인원 기회비용 날 지역 행정 군 극대 최대 지난해 기초자치단체 그동안 행정력 암투 글로벌 건국 폐쇄 도민 성향 광채 꿈 것 후 정책 위기 생명력 선진국 휘하 음모 정립 청산 간 다이 땅 국내 회한 소망 사무소 절망 배치 공직자 관행 개성관광 자치 짐 외교 일 몫 핵 퇴행 좌절 년 시 기축 선거 햇살 희망 번지 미증유 이 기업 발휘 해다 집행 결정 무엇 뒤 수 틀 해결 남북한 코드 금강산 전환점 극복 련 조화 대비 갈등 피 원년 운행 등 의 동해 면적 대결 육 부여 도정 혼란 단체장 직무 정치색 고난 경기 경협 국익 우리 활용 인재 나열 역량 떼 화두 기 새해 예산 불신 저력 국민 현상 단추 문제 과정 추진력 암울 올림픽 발굴 한뜻 때문 중요 단계 지 공유 대통령 한반도 처음 듯 태양 질주 적재적소 협의 청사진 인맥 달성 아침 대화 국호 북 성공 유념 마음 구축 인구 역사 생산 도전 미국 동기 정과 다음 불씨 번 도약 당위 책임 여건 단절 포함 이것 가치 약점 평창 시련
--- 문서 1 번 ---
사무 소외 수동 전폭 증원 시스템 운동 처우 도생 올해 단체 연합회 정기 모두 처장 경제 연구 쾌거 밀알 사업 감사 규정 최일선 도출 제공 성과 구성원 부지런 유기 협조 현재 체제 걸 복지 소 중국 발전 소통 소리 베이징 기대 혼신 년차 봉착 바탕 활동 종목 협의회 삶 부회장 우수 체육 전무 가운데 일조 개최 도 종목별 성원 정체 스포츠 체육회 장애 정호원 축년 개선 김흥수 

### Scikit-learn을 사용한 TF-IDF

- 한글자 단어 제외
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- Useful option: tokenizer, stop_words



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [ ]:
print(len(all_sentences))

20


In [ ]:
print(' '.join(all_sentences))

표출 도시 고유 지혜 한미 관계 지방 취임 오바마 민족 업그레이드 인사 얼마 올해 추진 과제 출발 단체 사람 슬기 후보 변화 열차 경제 평화 실행 기반 모색 조기 정치 성과 현안 쇠 어제 변수 자질 발전 구체 정부 낭비성 점 개성 기대 바람 활력 때 시간 자본 통일 유치 열쇠 몸 민 한국 논리 바다 바탕 근거 목적 당국 화해 자생력 정치력 전력 유능 방향 동계 자신 정상 방법 회생 속 중단 통행 도 증오 남북 공감대 강원도 미래 기억 축년 능력 흑인 파악 시대 욕 경의선 인원 기회비용 날 지역 행정 군 극대 최대 지난해 기초자치단체 그동안 행정력 암투 글로벌 건국 폐쇄 도민 성향 광채 꿈 것 후 정책 위기 생명력 선진국 휘하 음모 정립 청산 간 다이 땅 국내 회한 소망 사무소 절망 배치 공직자 관행 개성관광 자치 짐 외교 일 몫 핵 퇴행 좌절 년 시 기축 선거 햇살 희망 번지 미증유 이 기업 발휘 해다 집행 결정 무엇 뒤 수 틀 해결 남북한 코드 금강산 전환점 극복 련 조화 대비 갈등 피 원년 운행 등 의 동해 면적 대결 육 부여 도정 혼란 단체장 직무 정치색 고난 경기 경협 국익 우리 활용 인재 나열 역량 떼 화두 기 새해 예산 불신 저력 국민 현상 단추 문제 과정 추진력 암울 올림픽 발굴 한뜻 때문 중요 단계 지 공유 대통령 한반도 처음 듯 태양 질주 적재적소 협의 청사진 인맥 달성 아침 대화 국호 북 성공 유념 마음 구축 인구 역사 생산 도전 미국 동기 정과 다음 불씨 번 도약 당위 책임 여건 단절 포함 이것 가치 약점 평창 시련 사무 소외 수동 전폭 증원 시스템 운동 처우 도생 올해 단체 연합회 정기 모두 처장 경제 연구 쾌거 밀알 사업 감사 규정 최일선 도출 제공 성과 구성원 부지런 유기 협조 현재 체제 걸 복지 소 중국 발전 소통 소리 베이징 기대 혼신 년차 봉착 바탕 활동 종목 협의회 삶 부회장 우수 체육 전무 가운데 일조 개최 도 종목별 성원 정체 스포츠 체육회 장애 정호원 축년 개선 김흥수 폭 분담 확대 지역 이사 체전 군 지난해 기관 그동안 

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
    word2id[feature] = idx

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
tfidf_vectorizer.vocabulary_  ## vocabulary_dict: A mapping of terms to feature indices.

{'표출': 2410,
 '도시': 590,
 '고유': 183,
 '지혜': 2089,
 '한미': 2460,
 '관계': 231,
 '지방': 2059,
 '취임': 2266,
 '오바마': 1500,
 '민족': 802,
 '업그레이드': 1385,
 '인사': 1715,
 '얼마': 1383,
 '올해': 1515,
 '추진': 2239,
 '과제': 229,
 '출발': 2252,
 '단체': 508,
 '사람': 1017,
 '슬기': 1242,
 '후보': 2618,
 '변화': 896,
 '열차': 1454,
 '경제': 138,
 '평화': 2385,
 '실행': 1311,
 '기반': 355,
 '모색': 736,
 '조기': 1957,
 '정치': 1934,
 '성과': 1136,
 '현안': 2529,
 '어제': 1377,
 '변수': 895,
 '자질': 1796,
 '발전': 844,
 '구체': 278,
 '정부': 1915,
 '낭비성': 427,
 '개성': 76,
 '기대': 352,
 '바람': 812,
 '활력': 2591,
 '시간': 1250,
 '자본': 1780,
 '통일': 2322,
 '유치': 1617,
 '열쇠': 1452,
 '한국': 2454,
 '논리': 454,
 '바다': 811,
 '바탕': 817,
 '근거': 328,
 '목적': 749,
 '당국': 519,
 '화해': 2571,
 '자생력': 1783,
 '정치력': 1935,
 '전력': 1863,
 '유능': 1600,
 '방향': 867,
 '동계': 615,
 '자신': 1785,
 '정상': 1919,
 '방법': 853,
 '회생': 2605,
 '중단': 2023,
 '통행': 2326,
 '증오': 2043,
 '남북': 419,
 '공감대': 188,
 '강원도': 59,
 '미래': 781,
 '기억': 362,
 '축년': 2245,
 '능력': 485,
 '흑인': 2626,
 '파악': 2359,
 '시대': 1254

In [ ]:
tfidf_matrix.shape

(20, 2630)

In [ ]:
with open(os.path.join(data_path, 'news_sample.txt'), 'r', encoding='utf-8') as f:
  for idx, line in enumerate(f.readlines()):
    print(f"--- 문서 {idx} 번 ---")
    print(line[:200])

--- 문서 0 번 ---
새로운 희망 공유하고 새 출발하자 기축년(己丑年) 새해다.새로운 희망을 공유하고 새 출발을 다짐할 때다.짙푸른 동해바다를 뚫고 이글거리는 광채를 뿜으며 힘차게 솟아오르는 태양을 안는다. 어제와 다른 이 아침의 햇살을 받으며 왜 간절한 소망이 없겠는가.지난날의 갈등과 혼란 회한을 뒤로하고 새로 일어서야 한다.올해는 불신과 증오, 암투와 음모, 좌절과 절망 등
--- 문서 1 번 ---
<새해 이렇게 하겠습니다>스포츠 "최일선 현장에서 진솔한 소리 듣겠다" 최동용 도체육회 사무처장 지도자와 선수 등 체육의 최일선 현장에서 진솔한 소리를 더 많이 듣겠다. 강원체육이 지금까지 잘 할 수 있었던 것은 운동장 체육관 등 현장에서 지도자와 선수들이 묵묵히 운동한 결과다. 애로사항 등을 청취 체육정책에 반영하는 소통의 역할에 충실하겠다. 도청·도교육
--- 문서 2 번 ---
<새해 이렇게 하겠습니다>경제 '저탄소 녹색성장' 도모 전수산 도상공회의소협의회장 강원경제는 경제성장에 필요한 입지여건이나 기반조성이 타 지역보다 열악하고 최근 세계경제 침체의 영향으로 많은 어려움을 겪고 있다. 그러나 최근 수도권과의 교통망이 확충되면서 중앙 및 지방정부의 지역전략산업 육성 등 유리한 발전여건을 갖춰 나가고 있다. 정부의 5+2광역경제권 
--- 문서 3 번 ---
<새해 이렇게 하겠습니다>문화 이대근 강원도문화원연합회장 “지속적 향토사 연구로 우리 역사 되살리겠다” 어김없이 새로운 한 해가 밝았다. 지난해는 많은 시련을 안겨줬다. 선진국의 문턱에서 넘지 못하고 머뭇거리며 지나온 10년 세월이 원망스러운 가운데 이번에는 미국발 경제한파가 발목을 잡고 있다. 우리나라는 국민소득에서 보면 전 세계에서 24번째 국가에 머물
--- 문서 4 번 ---
<새해 이렇게 하겠습니다>속초 고성 양양 ■속초시 환동해 수출입항 메카 거듭 의정 옴부즈맨 제도 활성화 채용생 속초시장은 “올해는 민선4기를 사실상 마무리 짓는 해로 지금까지 성과를 바탕으로 성실한 마무리와 함께 경제

In [ ]:
for idx, sent in enumerate(all_sentences):
    print(f"--- 문서 {idx} 번 ---")
    results = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
    results.sort(key = lambda element : element[1], reverse=True)
    print(results)
    print("\n")

--- 문서 0 번 ---
[('한미', 0.08316005260563078), ('취임', 0.08316005260563078), ('오바마', 0.08316005260563078), ('열차', 0.08316005260563078), ('어제', 0.08316005260563078), ('자질', 0.08316005260563078), ('낭비성', 0.08316005260563078), ('개성', 0.08316005260563078), ('열쇠', 0.08316005260563078), ('근거', 0.08316005260563078), ('목적', 0.08316005260563078), ('당국', 0.08316005260563078), ('화해', 0.08316005260563078), ('정치력', 0.08316005260563078), ('유능', 0.08316005260563078), ('통행', 0.08316005260563078), ('증오', 0.08316005260563078), ('공감대', 0.08316005260563078), ('기억', 0.08316005260563078), ('흑인', 0.08316005260563078), ('경의선', 0.08316005260563078), ('기회비용', 0.08316005260563078), ('기초자치단체', 0.08316005260563078), ('암투', 0.08316005260563078), ('건국', 0.08316005260563078), ('폐쇄', 0.08316005260563078), ('광채', 0.08316005260563078), ('생명력', 0.08316005260563078), ('휘하', 0.08316005260563078), ('음모', 0.08316005260563078), ('정립', 0.08316005260563078), ('청산', 0.08316005260563078), ('다이', 0.08316005260563078), ('회한', 0.083160